In [14]:
# Call neccesary liraries
from Bio import PDB
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio import SeqIO
import numpy as np
import torch
import pydssp
import biolib

# Get PDB data
pdb_code = "2RH1"
pdb_file = PDB.PDBList()
pdb_file.retrieve_pdb_file(pdb_code, file_format="pdb")

Structure exists: 'C:\Users\Student\OneDrive - Aston University\Documents\Biology\Project\Project_automation\Python\rh\pdb2rh1.ent' 


'C:\\Users\\Student\\OneDrive - Aston University\\Documents\\Biology\\Project\\Project_automation\\Python\\rh\\pdb2rh1.ent'

In [15]:
# Create a dictionary for amino acid conversion 3 letter to one
three_to_one = {
    'ALA': 'A', 'CYS': 'C', 'ASP': 'D', 'GLU': 'E', 'PHE': 'F',
    'GLY': 'G', 'HIS': 'H', 'ILE': 'I', 'LYS': 'K', 'LEU': 'L',
    'MET': 'M', 'ASN': 'N', 'PRO': 'P', 'GLN': 'Q', 'ARG': 'R',
    'SER': 'S', 'THR': 'T', 'VAL': 'V', 'TRP': 'W', 'TYR': 'Y'
}

In [16]:
# Parse the file
parser = PDB.PDBParser(QUIET=True)
structure = parser.get_structure(pdb_code, f"rh\\pdb2rh1.pdb")

# Select the first structure in the file
model = structure[0]

# Select the A chain
chain = model['A']


In [4]:
# Retrieve coordinates
coordinates = []
fasta_seq = []
for residue in chain:
    if PDB.is_aa(residue):
        res_name = residue.get_resname()
        fasta_seq.append(three_to_one.get(res_name))
        for atom in residue:
            if atom.get_name() in ['N', 'CA', 'C', 'O']:
                coordinates.append(atom.coord)

In [29]:
print(len(fasta_seq))
print(len(coordinates))

442
1768


In [5]:
# Prepare eliments of pytorch tensor
L = sum(1 for residue in chain if PDB.is_aa(residue))
atoms = 4
xyz = 3

# Make coordinates into array first to increase efficiency
coord_array = np.array(coordinates)

# Create tensor
coord_tensor = torch.tensor(coord_array, dtype=torch.float32).reshape([L, atoms, xyz])

In [6]:
#dssp_hbond_matrix = pydssp.get_hbond_map(coord_tensor) > 0.5

# Use pydssp to get secondary structure
ss_data = pydssp.assign(coord_tensor, out_type='c3')
print(ss_data)

['-' '-' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H'
 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' '-' 'H' 'H' 'H'
 '-' '-' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H'
 'H' 'H' 'H' '-' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' '-' '-' '-' '-'
 '-' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H'
 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H'
 '-' '-' '-' '-' '-' '-' '-' '-' '-' '-' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H'
 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' '-' '-'
 '-' '-' 'E' '-' '-' '-' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' '-' '-' '-' '-'
 '-' '-' '-' '-' 'E' '-' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' '-'
 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H'
 'H' 'H' 'H' '-' '-' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H' '-' '-' '-' 'E' '-'
 'E' '-' 'E' 'E' '-' '-' '-' '-' '-' 'E' 'E' 'E' 'E' '-' '-' 'E' 'E' '-'
 'E' '-' '-' '-' '-' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H

In [7]:
ss_data_string = "".join(ss_data)
print(ss_data_string)

--HHHHHHHHHHHHHHHHHHHHHHHHHHHHHH-HHH--HHHHHHHHHHHHHHHHHHH-HHHHHHHHHH-----HHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHH----------HHHHHHHHHHHHHHHHHHHHHHHH----E---HHHHHHHH--------E-HHHHHHHHHHH-HHHHHHHHHHHHHHHHHHHHH--HHHHHHHH---E-E-EE-----EEEE--EE-E----HHHHHHHHHHHH------E--HHHHHHHHHHHHHHHHHHHHH-HHHHHHHHH--HHHHHHHHHHHHHHHHHHHH--HHHHHHHH---HHHHHHHH---HHHHH-HHHHHHHHHHHHH---HHH----HHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHH------HHHHHHHHHHHHHH---HHHHHHH-HHHHHHHHHH---


In [21]:
# Make fasta string for deepTMHMM
fasta_string = "".join(fasta_seq)

In [22]:
# Use deepTMHMM locally in python
biolib.utils.STREAM_STDOUT = True
deeptmhmm_job = biolib.deeptmhmm.cli(args=fasta_string, machine='local')
deeptmhmm_job.save_files('output')

AttributeError: module 'biolib' has no attribute 'deeptmhmm'

In [28]:
seq_record = SeqRecord(Seq(fasta_string), id="2rh1")

output_filename = "2rh1.fasta"  # You can name the file as you wish
with open(output_filename, "w") as output_file:
    SeqIO.write(seq_record, output_file, "fasta")

print(fasta_string)

DEVWVVGMGIVMSLIVLAIVFGNVLVITAIAKFERLQTVTNYFITSLACADLVMGLAVVPFGAAHILMKMWTFGNFWCEFWTSIDVLCVTASIETLCVIAVDRYFAITSPFKYQSLLTKNKARVIILMVWIVSGLTSFLPIQMHWYRATHQEAINCYAEETCCDFFTNQAYAIASSIVSFYVPLVIMVFVYSRVFQEAKRQLNIFEMLRIDEGLRLKIYKDTEGYYTIGIGHLLTKSPSLNAAKSELDKAIGRNTNGVITKDEAEKLFNQDVDAAVRGILRNAKLKPVYDSLDAVRRAALINMVFQMGETGVAGFTNSLRMLQQKRWDEAAVNLAKSRWYNQTPNRAKRVITTFRTGTWDAYKFCLKEHKALKTLGIIMGTFTLCWLPFFIVNIVHVIQDNLIRKEVYILLNWIGYVNSGFNPLIYCRSPDFRIAFQELLCL


In [17]:
# Check if DeepTMHMM has worked from results produced in terminal

# Define TMH predicted regions
tmh_regions = [
    (5, 30), (43, 64), (80, 101), (122, 143),
    (170, 191), (375, 395), (407, 427)
]

# Extract residues
tmh_residues = []
for start, end in tmh_regions:
    for residue in chain:
        res_id = residue.get_id()[1]
        if start <= res_id <= end:
            res = residue.get_resname()
            ca_atom = residue['CA'].coord if 'CA' in residue else None
            tmh_residues.append((res_id, res, ca_atom))

# Unpack and format tmh_residues for readability
for res_id, res_name, ca_coord in tmh_residues:
    print(f"Residue {res_id}: {res_name}, Coordinates: {ca_coord}")


Residue 29: ASP, Coordinates: [-51.922  -2.262  22.148]
Residue 30: GLU, Coordinates: [-53.821  -1.515  18.887]
Residue 43: ILE, Coordinates: [-46.85   16.274  12.593]
Residue 44: VAL, Coordinates: [-47.195  15.967   8.801]
Residue 45: LEU, Coordinates: [-49.954  18.638   8.719]
Residue 46: ALA, Coordinates: [-47.783  20.933  10.893]
Residue 47: ILE, Coordinates: [-44.751  20.613   8.596]
Residue 48: VAL, Coordinates: [-46.721  21.051   5.36 ]
Residue 49: PHE, Coordinates: [-48.951  23.916   6.654]
Residue 50: GLY, Coordinates: [-46.193  26.04    8.158]
Residue 51: ASN, Coordinates: [-43.713  25.636   5.273]
Residue 52: VAL, Coordinates: [-46.453  26.245   2.687]
Residue 53: LEU, Coordinates: [-47.156  29.441   4.688]
Residue 54: VAL, Coordinates: [-43.458  30.517   4.563]
Residue 55: ILE, Coordinates: [-43.164  29.906   0.791]
Residue 56: THR, Coordinates: [-46.403  31.77    0.067]
Residue 57: ALA, Coordinates: [-45.368  34.762   2.263]
Residue 58: ILE, Coordinates: [-41.96   35.25   